In [1]:
import sys
assert sys.version_info >= (3, 6), "Sonnet 2 requires Python >=3.6"

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tree
import pandas as pd

try:
  import sonnet.v2 as snt
  tf.enable_v2_behavior()
except ImportError:
  import sonnet as snt

print("TensorFlow version {}".format(tf.__version__))
print("Sonnet version {}".format(snt.__version__))

TensorFlow version 2.3.1
Sonnet version 2.0.0


In [3]:
sys.path.append('/Users/tuckerkj/git/quantumML/python/models/')
import vae

In [ ]:
import importlib
importlib.reload(vae)

# t = 0

In [6]:
is_data = pd.read_table('/Users/tuckerkj/python/data/tfi_n10_upx/tfi_n10_upx_t0.txt', delimiter=' ', usecols=range(10)).values
is_train = is_data[0:80000]
is_test = is_data[80001:100000]

In [7]:
original_dim = is_train.shape[1]

In [8]:
# Get data sliced for SGD
batch_size = 128

train_dataset = (
    tf.data.Dataset.from_tensor_slices(is_train)
    .shuffle(1000)
    .repeat(-1)  # repeat indefinitely
    .batch(batch_size, drop_remainder=True)
    .prefetch(-1))

valid_dataset = (
    tf.data.Dataset.from_tensor_slices(is_test)
    .repeat(1)  # 1 epoch
    .batch(batch_size)
    .prefetch(-1))

In [7]:
# network parameters
learning_rate = 3e-4

# 50K training steps unless otherwise noted
# F does not appear to be impacted up to four digits by the sampling procedure
# used to reconstruct the state

# F = 0.9873 (larger training set) (no beta) (Loss: 4.582048416137695 Recon loss: 2.8563406467437744)
intermediate_dim = [100]
latent_dim = 4

optimizer = snt.optimizers.Adam(learning_rate=learning_rate)

enc = vae.Encoder(intermediate_dim, latent_dim)
dec = vae.Decoder(intermediate_dim, original_dim)
qstvae = vae.VAE(enc, dec)

@tf.function
def train_step(data, beta):
    with tf.GradientTape() as tape:
        model_output = qstvae(tf.cast(data, tf.float32))#, beta)
    
    trainable_variables = qstvae.trainable_variables
    grads = tape.gradient(model_output['loss'], trainable_variables)
    optimizer.apply(grads, trainable_variables)
    
    return model_output

In [13]:
# Train
num_training_updates = 50000

train_losses = []
recon_losses = []
for step_index, data in enumerate(train_dataset):
    #beta = tf.constant(0.85*(step_index/float(num_training_updates)))
    beta = 1.0
    train_results = train_step(data, beta)
    train_losses.append(train_results['loss'])
    recon_losses.append(train_results['x_recon_loss'])
    
    if (step_index + 1) % 1000 == 0:
        print('%d loss: %f recon loss: %f' % (step_index+1, np.mean(train_losses[-100:]), np.mean(recon_losses[-100:])))
        
    if (step_index + 1) % num_training_updates == 0:
        break

1000 loss: 6.953280 recon loss: 6.880973
2000 loss: 6.948987 recon loss: 6.890866
3000 loss: 6.944779 recon loss: 6.897463
4000 loss: 6.940837 recon loss: 6.902632
5000 loss: 6.941781 recon loss: 6.908838
6000 loss: 6.937864 recon loss: 6.910003
7000 loss: 6.938973 recon loss: 6.911843
8000 loss: 6.940070 recon loss: 6.915005
9000 loss: 6.939018 recon loss: 6.915903
10000 loss: 6.938797 recon loss: 6.913396
11000 loss: 6.938328 recon loss: 6.916503
12000 loss: 6.938834 recon loss: 6.917307
13000 loss: 6.939315 recon loss: 6.917589
14000 loss: 6.935915 recon loss: 6.915759
15000 loss: 6.937923 recon loss: 6.920103
16000 loss: 6.937616 recon loss: 6.918606
17000 loss: 6.937326 recon loss: 6.921682
18000 loss: 6.937371 recon loss: 6.920550
19000 loss: 6.935030 recon loss: 6.919858
20000 loss: 6.937127 recon loss: 6.921346
21000 loss: 6.936165 recon loss: 6.921560
22000 loss: 6.937075 recon loss: 6.923837
23000 loss: 6.937419 recon loss: 6.923038
24000 loss: 6.935955 recon loss: 6.922118
2

In [14]:
# Look at validation set
model_output = qstvae(tf.cast(is_test, dtype=tf.dtypes.float32))
print('Loss: {} Recon loss: {}'.format(model_output['loss'], model_output['x_recon_loss']))

Loss: 6.9346699714660645 Recon loss: 6.927407741546631


In [8]:
def bit_array(a):
    aa = []
    for c in a:
        if c == '0':
            aa.append(0)
        else:
            aa.append(1)
        
    return np.array(aa)

def bin_to_dec(b):
    dec = 0
    for idx, val in enumerate(b):
        dec += val << (len(b) - idx - 1)
        
    return dec

def update_counts2(psi, model, batch_size, per_batch_size):
    # Sample from the prior distribution on the latent space and run through the decoder
    latent_dim = model.decoder.hidden[0].input_size
    z = tf.random.normal([batch_size, latent_dim], mean=0.0, stddev=1.0, dtype=tf.dtypes.float32)
    output = tf.nn.sigmoid(model.decoder(z))

    # Sample from the binary distributions coming out of the decoder to get spins
    all_meas_int = []
    for idx in range(per_batch_size):
        eps = tf.random.uniform(output.shape, minval=0, maxval=1, dtype=tf.dtypes.float32)
        all_meas_int.append(tf.cast(tf.math.greater_equal(output, eps), tf.int32).numpy())
    meas_int = tf.concat(all_meas_int, axis=0).numpy()
    
    for ii in range(meas_int.shape[0]):
        idx = bin_to_dec(meas_int[ii,:])
        psi[idx] += 1

def get_hist(n, meas_int):
    psi = np.zeros(2**n)
    for ii in range(meas_int.shape[0]):
        idx = bin_to_dec(meas_int[ii,:])
        psi[idx] += 1
        
    # Normalize
    psi = np.sqrt(psi*(1.0/float(meas_int.shape[0])))
    
    return psi
        
def update_counts(psi, model, batch_size):
    meas_int = model.sample(batch_size).numpy()
    
    for ii in range(meas_int.shape[0]):
        idx = bin_to_dec(meas_int[ii,:])
        psi[idx] += 1

def get_psi(model, num_samples):
    n = model.encoder.hidden[0].input_size
    psi = np.zeros(2**n)
    batch_size = 1000
    per_batch_size = 10
    total_samples = 0
    while total_samples < num_samples:
        update_counts(psi, model, batch_size)
        #update_counts2(psi, model, batch_size, per_batch_size)
        total_samples = total_samples + batch_size
        
        if total_samples % 100000 == 0:
            print(total_samples)
        
    # Normalize
    psi = np.sqrt(psi*(1.0/float(total_samples)))
    #psi = np.sqrt(psi*(1.0/float(total_samples*per_batch_size)))
    
    return psi

import math
def get_psi_loss(model, num_samples):
    n = model.encoder.hidden[0].input_size
    norm = 0
    psi = []
    for d in range(2**n):
        dbin = bit_array(np.binary_repr(d, width=n))
        dbin_input = np.tile(dbin, (num_samples,1))
        model_output = model(dbin_input.astype(float))
        val = np.exp(-0.5*model_output['loss'])
        psi.append(val)
        norm = norm + val*val
    norm = math.sqrt(norm)
    
    for ii in range(len(psi)):
        psi[ii] = psi[ii]/norm
        
    return np.array(psi)

In [16]:
psi = get_psi(qstvae, 1000000)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000


In [17]:
np.dot(psi, psi)

1.0

In [18]:
psi

array([0.0315119 , 0.03290897, 0.03168596, ..., 0.03098387, 0.02966479,
       0.02993326])

In [19]:
# Save the wave function
np.savetxt('/Users/tuckerkj/python/data/vae_tfi_n10_upx_t0.txt', psi)

# t = 0.1

In [11]:
is_data = pd.read_table('/Users/tuckerkj/python/data/tfi_n10_upx/tfi_n10_upx_t1.txt', delimiter=' ', usecols=range(10)).values
is_train = is_data[0:80000]
is_test = is_data[80001:100000]

In [12]:
original_dim = is_train.shape[1]

In [13]:
# Get data sliced for SGD
batch_size = 128

train_dataset = (
    tf.data.Dataset.from_tensor_slices(is_train)
    .shuffle(1000)
    .repeat(-1)  # repeat indefinitely
    .batch(batch_size, drop_remainder=True)
    .prefetch(-1))

valid_dataset = (
    tf.data.Dataset.from_tensor_slices(is_test)
    .repeat(1)  # 1 epoch
    .batch(batch_size)
    .prefetch(-1))

In [16]:
# Train
num_training_updates = 50000

train_losses = []
recon_losses = []
for step_index, data in enumerate(train_dataset):
    #beta = tf.constant(0.85*(step_index/float(num_training_updates)))
    beta = 1.0
    train_results = train_step(data, beta)
    train_losses.append(train_results['loss'])
    recon_losses.append(train_results['x_recon_loss'])
    
    if (step_index + 1) % 1000 == 0:
        print('%d loss: %f recon loss: %f' % (step_index+1, np.mean(train_losses[-100:]), np.mean(recon_losses[-100:])))
        
    if (step_index + 1) % num_training_updates == 0:
        break

1000 loss: 6.475722 recon loss: 6.450878
2000 loss: 6.479484 recon loss: 6.441965
3000 loss: 6.474957 recon loss: 6.438721
4000 loss: 6.459610 recon loss: 6.412510
5000 loss: 6.457504 recon loss: 6.403729
6000 loss: 6.465859 recon loss: 6.384759
7000 loss: 6.465143 recon loss: 6.343863
8000 loss: 6.460498 recon loss: 6.311026
9000 loss: 6.447575 recon loss: 6.271127
10000 loss: 6.444769 recon loss: 6.240985
11000 loss: 6.454764 recon loss: 6.241408
12000 loss: 6.458661 recon loss: 6.223857
13000 loss: 6.455796 recon loss: 6.210427
14000 loss: 6.451104 recon loss: 6.202951
15000 loss: 6.443942 recon loss: 6.185785
16000 loss: 6.451401 recon loss: 6.180143
17000 loss: 6.456146 recon loss: 6.177778
18000 loss: 6.451734 recon loss: 6.167236
19000 loss: 6.447408 recon loss: 6.167718
20000 loss: 6.441145 recon loss: 6.155897
21000 loss: 6.451897 recon loss: 6.157735
22000 loss: 6.453855 recon loss: 6.152004
23000 loss: 6.444924 recon loss: 6.132488
24000 loss: 6.446603 recon loss: 6.137098
2

In [17]:
psi = get_psi(qstvae, 1000000)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000


In [18]:
# Save the wave function
np.savetxt('/Users/tuckerkj/python/data/vae_tfi_n10_upx_t1.txt', psi)

In [9]:
psi = get_hist(original_dim, is_train)

In [10]:
# Save the histogram
np.savetxt('/Users/tuckerkj/python/data/hist_tfi_n10_upx_t1.txt', psi)